In [16]:
#Import Libraries

import numpy as np
import pandas as pd
import missingno as msno  
import plotly.express as px

In [23]:
# Supress Scientific Notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [17]:
#Loading Data
df = pd.read_csv('/content/kc_house_data.csv')

In [4]:
#fazer a conversão de date 
df['date'] = pd.to_datetime(df['date'])

**OLHAR O MAPA E CONSEGUIR IDENTIFICAR AS CASAS COM O MAIOR PREÇO**

In [5]:
df = pd.read_csv('/content/kc_house_data.csv')

houses = df[['id','lat','long','price']].copy()
import plotly.express as px

for i in range( len( houses) ):
    if houses.loc[i,'price']<= 321950:
        houses.loc[i,'level'] = 0

    elif (houses.loc[i,'price'] > 321950)  & ( houses.loc[i,'price'] <= 450000):
         houses.loc[i,'level'] = 1

    elif ( houses.loc[i,'price'] > 450000)  & ( houses.loc[i,'price'] <= 645000):
         houses.loc[i,'level'] = 2
    else:
         houses.loc[i,'level'] = 3
 

houses['level'] = houses['level'].astype( int )
    
    
fig = px.scatter_mapbox( houses,
                         lat="lat",
                         lon="long",
                         color="level",
                         size="price",
                         color_continuous_scale=px.colors.cyclical.IceFire,
                                    size_max=15,
                                    zoom=10)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()




**FILTRO INTERATIVO**

In [30]:
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import fixed

df = pd.read_csv('/content/kc_house_data.csv')

df['is_waterfront'] = df['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no')

df['level'] = df['price'].apply( lambda x: 0 if x< 321950 else
                                           1 if ( x > 321950) & (x < 450000 )else
                                           2 if ( x > 450000) & (x < 645000 )else 3 )

df['level'] = df['level'].astype( int )
style = {'description_width': 'initial'}

#interative buttons
price_limit = widgets.IntSlider(
    value =  540000,
    min = 75000,
    max = 77000000,
    step = 1, 
    description= 'Maximun Price',
    disable=False,
    style = style
)

waterfront_bar = widgets.Dropdown(
    options= df['is_waterfront'].unique().tolist(),
    value='yes',
    description='Water View',
    disable=False)


def update_map( df, waterfront, limit ):
    houses = df[(df['price'] <= limit) & 
            (df['is_waterfront'] == waterfront)][['id', 'lat', 'long', 'price', 'level', 'bathrooms']]

fig = px.scatter_mapbox( houses,
                          lat="lat",
                          lon="long",
                          color="level",
                          size="price",
                          color_continuous_scale= px.colors.cyclical.HSV,
                          size_max=15,
                          zoom=10 )

fig.update_layout(mapbox_style="open-street-map" )
fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0} )
fig.show()

widgets.interactive( update_map, df=fixed( df ), waterfront=waterfront_bar, limit=price_limit)


interactive(children=(Dropdown(description='Water View', index=1, options=('no', 'yes'), value='yes'), IntSlid…

**Faça um gráfico com o tamanho dos pontos sendo igual ao tamanho da sala de estar**

In [7]:
import plotly.express as px # biblioteca usada para criar mapas

fig_map = df[['id', 'lat', 'long', 'price', 'sqft_living']]

# CRIANDO MAPA
maps = px.scatter_mapbox(fig_map,
                         lat= 'lat',
                         lon= 'long',
                         color= 'price',
                         color_continuous_scale= px.colors.cyclical.HSV,
                         size= 'sqft_living',
                         size_max= 15,
                         zoom = 10 )

maps.update_layout (mapbox_style= 'open-street-map')
maps.update_layout (height= 1080, margin= {'r': 0, 't': 0, 'l': 0, 'b': 0 })

maps.show()



Qual a média do preço de compra dos imóveis por “Nível”? 
- Nível 0 -> Preço entre R$ 0
e R$ 321.950 - Nível 1 -> Preço entre R$ 321.950 e R$ 450.000 - Nível 2 -> Preço entre R$
450.000 e R$ 645.000 - Nível 3 -> Acima de R$ 645.000

2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ? - Size 0 -> Tamanho
entre 0 e 1427 sqft - Size 1 -> Tamanho entre 1427 e 1910 sqft - Size 2 -> Tamanho entre
1910 e 2550 sqft - Size 3 -> Tamanho acima de 2550 sqft

3. Adicione as seguinte informações ao conjunto de dados original: - Place ID: Identificação da
localização - OSM Type: Open Street Map type - Country: Nome do País - Country Code:
Código do País

4. Adicione os seguinte filtros no Mapa: - Tamanho mínimo da área da sala de estar. - Número
mínimo de banheiros. - Valor Máximo do Preço. - Tamanho máximo da área do porão. -
Filtro das Condições do Imóvel. - Filtro por Ano de Construção.

5. Adicione os seguinte filtros no Dashboard: - Filtro por data disponível para compra. - Filtro
por ano de renovação. - Filtro se possui vista para a água ou não.


In [10]:
#import libraries

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import gridspec
from matplotlib import pyplot as plt
import ipywidgets as widgets
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual


In [11]:
# Supress Scientific Notation
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.2f}'.format)


Qual a média do preço de compra dos imóveis por “Nível”?
- Nível 0 -> Preço entre R$ 0 e R$ 321.950
- Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
- Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
- Nível 3 -> Acima de R$ 645.000

In [12]:
df['level'] = df['price'].apply( lambda x: 'lv0' if x <= 321950 else
                                           'lv1' if ( x > 321950 ) & ( x <= 450000 ) else
                                           'lv2' if ( x > 450000 ) & ( x <= 645000 ) else 'lv3' )
# final dataset
df = df[['level', 'price']].groupby( 'level' ).mean().reset_index()
df

,level,price
0,lv0,251557.64989
1,lv1,385688.68150
2,lv2,539730.96001
3,lv3,987540.22446


Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft


In [18]:
df['size'] = df['sqft_living'].apply( lambda x: 'size0' if x <= 1427 else
                                                    'size1' if ( x > 1427 ) & ( x <= 1910 ) else
                                                    'size2' if ( x > 1910 ) & ( x <= 2550 ) else 'size3' )
# final dataset
df = df[['size', 'sqft_living']].groupby( 'size' ).mean().reset_index()
df

,size,sqft_living
0,size0,1123.83253
1,size1,1664.96233
2,size2,2211.79279
3,size3,3329.60702


Adicione as seguinte informações ao conjunto de dados original:
- Place ID: Identificação da localização
- OSM Type: Open Street Map type
- Country: Nome do País
- Country Code: Código do País

Adicione os seguinte filtros no Mapa:
- Tamanho mínimo da área da sala de estar.
- Número mínimo de banheiros.
- Valor Máximo do Preço.
- Tamanho máximo da área do porão.
- Filtro das Condições do Imóvel.
- Filtro por Ano de Construção.


In [32]:
df['is_waterfront']= df['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )

# define level of prices
df['level'] = df['price'].apply( lambda x: 0 if x <= 321950 else 1 
                                             if ( x > 321950 ) & ( x <= 450000) else 2
                                             if ( x > 450000 ) & ( x <= 645000) else 3 )
df['level'] = df['level'].astype( int )
# Iteractive buttons
price_limit = widgets.IntSlider(
value = 540000,
min = 75000,
max = 7700000,
step = 1,
description='Maximun Price',
disable=False,
style={'description_width': 'initial'}
)

waterfront_bar = widgets.Dropdown(
options = df['is_waterfront'].unique().tolist(),
value = 'yes',
description = 'Water View',
disable=False)
livingroom_limit = widgets.IntSlider(
value = int( df['sqft_living'].mean() ),
min = df['sqft_living'].min(),
max = df['sqft_living'].max(),
step = 1,
description='Minimum Living Room Size',
disable=False,
style={'description_width': 'initial'}
)
bathroom_limit = widgets.IntSlider(
value = int( df['bathrooms'].mean() ),
min = df['bathrooms'].min(),
max = df['bathrooms'].max(),
step = 1,
description='Minimum Bathroom Number',
disable=False,
style={'description_width': 'initial'}
)
basement_limit = widgets.IntSlider(
value = int( df['sqft_basement'].mean() ),
min = df['sqft_basement'].min(),
max = df['sqft_basement'].max(),
step = 1,
description='Minimum Basement Size',
disable=False,
style={'description_width': 'initial'}
)
condition_limit = widgets.IntSlider(
value = int( df['condition'].mean() ),
min = df['condition'].min(),
max = df['condition'].max(),
step = 1,
description='Minimum condition',
disable=False,
style={'description_width': 'initial'}
)
year_limit = widgets.IntSlider(
value = int( df['yr_built'].mean() ),
min = df['yr_built'].min(),
max = df['yr_built'].max(),
step = 1,
description='Year Built',
disable=False,
style={'description_width': 'initial'}
)
def update_map( df, waterfront, limit, livingroom_limit, bathroom_limit,basement_limit, condition_limit, year_limit ):
             houses = df[(df['price'] <= limit) &
            (df['is_waterfront'] == waterfront) &
            (df['sqft_living'] >= livingroom_limit) &
            (df['bathrooms'] >= bathroom_limit) &
            (df['sqft_basement'] >= basement_limit) &
            (df['condition'] >= condition_limit) &
            (df['yr_built'] >= year_limit )][['id', 'lat', 'long', 'price','level']]
fig = px.scatter_mapbox( houses,
lat="lat",
lon="long",

size="price",
color="level",
color_continuous_scale=px.colors.cyclical.IceFire,
size_max=15,
zoom=10)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

widgets.interactive( update_map,
df=fixed( df ),
waterfront=waterfront_bar,
limit=price_limit,
livingroom_limit=livingroom_limit,
bathroom_limit=bathroom_limit,
basement_limit=basement_limit,
condition_limit=condition_limit,
year_limit=year_limit
)

interactive(children=(Dropdown(description='Water View', index=1, options=('no', 'yes'), value='yes'), IntSlid…